In [1]:
import pymupdf
from typing import Dict
import os

def load_document(file_path:str)->dict:
    if not os.path.exists(file_path):
        print(f'File Not Exists. Check the File!')
        raise ValueError
    file_name = file_path.split('\\')[-1].split('.')[0]
    # print(file_name)
    docs = pymupdf.open(file_path)
    document = []
    # print(docs.metadata)
    for page in docs:
        document.append(
            {
                "doc_content" : page.get_text(),
                "doc_id" : f"doc_{page.number + 1}",
                "doc_name" : file_name,
                "chunks" : []
            }
        )
    return document

def chunk_document(document, split_interval: int, overlap_ratio: float = 0.2) -> list[dict]:
    if not document:
        print("No document detected!!!")
        return
    
    for page in document:
        content = page['doc_content']
        
        # 'chunks' 필드가 없는 경우 초기화
        if 'chunks' not in page:
            page['chunks'] = []
        
        # 중첩 크기 계산
        overlap_size = int(split_interval * overlap_ratio)

        # 분할할 필요가 없을 때
        if split_interval >= len(content):
            chunk = content
            page['chunks'].append(
                {
                    'chunk_content': chunk,
                    'chunk_id': page['doc_id'] + '_chunk_1'
                }
            )
        else:
            i = 0
            start_idx = 0
            end_idx = split_interval
            
            while start_idx < len(content):
                chunk = content[start_idx:end_idx]
                page['chunks'].append(
                    {
                        'chunk_content': chunk,
                        'chunk_id': page['doc_id'] + f'_chunk_{i + 1}'
                    }
                )
                
                # 다음 청크의 시작 위치와 끝 위치를 중첩 크기를 반영하여 설정
                start_idx = end_idx - overlap_size
                end_idx = start_idx + split_interval
                
                i += 1

    return document


In [2]:
file_path = r'..\crawler\downloads\attachments\samilpwc_ai-business-use-cases.pdf'
document = load_document(file_path)
chunk_doc = chunk_document(document, split_interval=400)
# for doc in chunk_doc:
#     for chunk in doc['chunks']:
#         print('#'* 10)
#         print(f'{chunk}\n')

In [3]:
doc_count = 0
chunk_count = 0
for doc in chunk_doc:
    doc_count += 1
    for chunk in doc['chunks']:
        chunk_count += 1
print(f'Document 페이지 수 : {doc_count} // Chunk 수 : {chunk_count}')

Document 페이지 수 : 69 // Chunk 수 : 312


In [4]:
import os
import pickle
import json
import numpy as np
from google import generativeai as genai
from typing import List, Dict, Any
from tqdm import tqdm
import time
import google

class ContextualVectorDB:
    def __init__(self, name: str,  google_api_key=None):
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./sample/{name}/contextual_vector_db.pkl"
        self.google_api_key = os.environ['GOOGLE_API_KEY']
        genai.configure(api_key=self.google_api_key)

    def situate_context(self, doc:str, chunk:str)->tuple[str, Any]:
        DOCUMENT_CONTEXT_PROMPT = """
        <document>
        {doc_content}
        </document>
        """

        CHUNK_CONTEXT_PROMPT = """
        Here is the chunk we want to situate within the whole document
        <chunk>
        {chunk_content}
        </chunk>

        Please give a short succint context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
        Answer only with the succint context in Korean(한국어) and nothing else.
        Remember the Answer should be in Korean(Hangul)
        """
        model_name = "gemini-1.5-flash"        
        model = genai.GenerativeModel(model_name)
        try:
            response = model.generate_content(
                f"""
                {DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc)}\n
                {CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk)}
                """
            )
            return response.text
        except google.api_core.exceptions.ResourceExhausted as re:
            time.sleep(60)
            try:
                response = model.generate_content(
                    f"""
                    {DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc)}\n
                    {CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk)}
                    """
                )
                return response.text
            except:
                return "No Contextual Information"
        except ValueError as ve:
            return 'No Contextual Information'
        
    def load_data(self, dataset: List[Dict[str, Any]], parallel_threads: int = 1):
        if self.embeddings and self.metadata:
            print("Vector database is already loaded. Skipping data loading.")
            return
        if os.path.exists(self.db_path):
            print("Loading vector database from disk.")
            self.load_db()
            return

        texts_to_embed = []
        metadata = []
        total_chunks = sum(len(doc['chunks']) for doc in dataset)
        
        def process_chunk(doc, chunk):
            contextual_text = self.situate_context(doc['doc_content'], chunk['chunk_content'])
            return {
                'texts_to_embed' : f"{chunk['chunk_content']} \n\n{contextual_text}",
                "metadata" : {
                    "doc_id" : doc['doc_id'],
                    "chunk_id" : chunk['chunk_id'],
                    "chunk_content" : chunk['chunk_content'],
                    "contextualized_content" : contextual_text
                }
            }
        
        futures = []
        with tqdm(total=total_chunks, desc="Generating Contextual") as pba:
            for doc in dataset:
                for chunk in doc['chunks']:
                    futures.append(process_chunk(doc, chunk))
                    pba.update(1)
        
        for future in tqdm(futures, total=total_chunks, desc = 'Processing Chunks'):
            texts_to_embed.append(future['texts_to_embed'])
            metadata.append(future['metadata'])

        self._embed_and_store(texts_to_embed, metadata)
        self.save_db()

    def _embed_and_store(self, texts: List[str], data: List[Dict[str, Any]]):
        batch_size = 10
        with tqdm(total = len(texts), desc = 'Embedding Contextual Chunks') as pbar:
            result = []
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i+batch_size]
                batch_result = genai.embed_content(
                    model = 'models/text-embedding-004',
                    content = batch
                )
                result.extend(batch_result['embedding'])
                pbar.update(len(batch))
        self.embeddings = result
        self.metadata = data

    def search(self, query: str, k: int = 20) -> List[Dict[str, Any]]:
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        else:
            try:
                query_embedding = genai.embed_content(
                    model = 'models/text-embedding-004',
                    content = query
                )['embedding']
            except:
                time.sleep(60)
                print(f'Time Sleep for 1 minute (Query Embedding)')
                query_embedding = genai.embed_content(
                    model = 'models/text-embedding-004',
                    content = query
                )['embedding']
            self.query_cache[query] = query_embedding

        if not self.embeddings:
            raise ValueError("No data loaded in the vector database.")
        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1][:k]
        
        top_results = []
        for idx in top_indices:
            result = {
                "metadata": self.metadata[idx],
                "similarity": float(similarities[idx]),
            }
            top_results.append(result)
        return top_results

    def save_db(self):
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)
        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)
        print("Successfully Save Contextual VectorDB")

    def load_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError("Vector database file not found. Use load_data to create a new database.")
        with open(self.db_path, "rb") as file:
            data = pickle.load(file)
        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"] 
        self.query_cache = json.loads(data["query_cache"])

In [5]:
contextual_db = ContextualVectorDB('test_2')
contextual_db.load_data(chunk_doc)

Loading vector database from disk.


In [6]:
# with open('./sample/test_1/contextual_vector_db.pkl', 'rb') as file:
#     data = pickle.load(file)

In [7]:
len(contextual_db.metadata)

312

In [8]:
import os
import json
from typing import List, Dict, Any
from tqdm import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

## Docker 네트워크 생성 (Optional) ##
# docker network create elastic

## Elasticsearch Docker 이미지 Pull ##
# docker pull docker.elastic.co/elasticsearch/elasticsearch:8.15.3

## Elasticsearch container 생성 및 시작 ##
# docker run -d --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" elasticsearch:8.8.0

## Docker 재시작 ##
# docker container restart elasticsearch

class ElasticsearchBM25:
    def __init__(self, index_name: str = "contextual_bm25_index_pwc"):
        self.es_client = Elasticsearch("http://localhost:9200")
        self.index_name = index_name
        # self.delete_index()
        self.create_index()

    def create_index(self):
        index_settings = {
            "settings": {
                "analysis": {"analyzer": {"my_nori": {"type": "custom","tokenizer": "nori_tokenizer","decompound_mode": "mixed"}}},
                "similarity": {"default": {"type": "BM25"}},
                "index.queries.cache.enabled": False  # Disable query cache
            },
            "mappings": {
                "properties": {
                    "chunk_content": {"type": "text", "analyzer": "my_nori"},
                    "contextualized_content": {"type": "text", "analyzer": "my_nori"},
                    "doc_id": {"type": "keyword", "index": False},
                    "chunk_id": {"type": "keyword", "index": False}
                }
            },
        }
        if not self.es_client.indices.exists(index=self.index_name):
            self.es_client.indices.create(index=self.index_name, body=index_settings)
            print(f"Created index: {self.index_name}")

    def delete_index(self):    
        if self.es_client.indices.exists(index=self.index_name):
            self.es_client.indices.delete(index=self.index_name)
            print(f"Deleted Elasticsearch index: {self.index_name}")

    def index_documents(self, documents: List[Dict[str, Any]]):
        actions = [
            {
                "_index": self.index_name,
                "_source": {
                    "chunk_content": doc["chunk_content"],
                    "contextualized_content": doc["contextualized_content"],
                    "doc_id": doc["doc_id"],
                    "chunk_id": doc["chunk_id"],
                },
            }
            for doc in documents
        ]
        success, _ = bulk(self.es_client, actions)
        self.es_client.indices.refresh(index=self.index_name)
        return success

    def search(self, query: str, k: int = 20) -> List[Dict[str, Any]]:
        self.es_client.indices.refresh(index=self.index_name)  # Force refresh before each search
        search_body = {
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["chunk_content", "contextualized_content"],
                }
            },
            "size": k,
        }
        response = self.es_client.search(index=self.index_name, body=search_body)
        return [
            {
                "doc_id": hit["_source"]["doc_id"],
                "chunk_id": hit["_source"]["chunk_id"],
                "chunk_content": hit["_source"]["chunk_content"],
                "contextualized_content": hit["_source"]["contextualized_content"],
                "score": hit["_score"],
            }
            for hit in response["hits"]["hits"]
        ]
    
def create_elasticsearch_bm25_index(db: ContextualVectorDB):
    es_bm25 = ElasticsearchBM25()
    es_bm25.index_documents(db.metadata)
    return es_bm25

def retrieve_advanced(query: str, db: ContextualVectorDB, es_bm25: ElasticsearchBM25, k: int, semantic_weight: float = 0.8, bm25_weight: float = 0.2):
    num_chunks_to_recall = 150

    semantic_results = db.search(query, k=num_chunks_to_recall)
    ranked_chunk_ids = [result['metadata']['chunk_id'] for result in semantic_results]

    bm25_results = es_bm25.search(query, k=num_chunks_to_recall)
    ranked_bm25_chunk_ids = [result['chunk_id'] for result in bm25_results]

    chunk_ids = list(set(ranked_chunk_ids + ranked_bm25_chunk_ids))
    
    chunk_id_to_score = {}
    
    for chunk_id in chunk_ids:
        score = 0
        if chunk_id in ranked_chunk_ids:
            index = ranked_chunk_ids.index(chunk_id)
            score += semantic_weight * (1 / (index + 1))  # Weighted 1/n scoring for semantic
        if chunk_id in ranked_bm25_chunk_ids:
            index = ranked_bm25_chunk_ids.index(chunk_id)
            score += bm25_weight * (1 / (index + 1))  # Weighted 1/n scoring for BM25
        chunk_id_to_score[chunk_id] = score

    # Sort chunk IDs by their scores in descending order
    sorted_chunk_ids = sorted(
        chunk_id_to_score.keys(), key=lambda x: (chunk_id_to_score[x], x[0], x[1]), reverse=True
    )

    # Assign new scores based on the sorted order
    for index, chunk_id in enumerate(sorted_chunk_ids):
        chunk_id_to_score[chunk_id] = 1 / (index + 1)

    # Prepare the final results
    final_results = []
    
    for chunk_id in sorted_chunk_ids[:k]:
        chunk_metadata = [chunk for chunk in db.metadata if chunk['chunk_id'] == chunk_id]
        is_from_semantic = chunk_id in ranked_chunk_ids
        is_from_bm25 = chunk_id in ranked_bm25_chunk_ids
        final_results.append({
            'chunk': chunk_metadata,
            'score': chunk_id_to_score[chunk_id],
            'from_semantic': is_from_semantic,
            'from_bm25': is_from_bm25
        })

    return final_results

In [10]:
es_bm25 = create_elasticsearch_bm25_index(contextual_db)

In [11]:
es = Elasticsearch("http://localhost:9200")
es.indices.refresh(index=es_bm25.index_name)
elasticsearch_results = es.search(
    index= es_bm25.index_name,
    
    body={
        "query": {
            "multi_match":
                    {
                        "query": "금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘",
                        "fields": ["chunk_content", "contextualized_content"]
                    }
                    },
        "size" : 5
        }
        )
elastic_rag = []

for id, result in enumerate(elasticsearch_results.body['hits']['hits']):
    print('#' * 25 + f' {id+1}번째 청크 ' + '#' * 25)
    print(f'{result['_source']['chunk_content']}')
    elastic_rag.append(result['_source']['chunk_content'])

######################### 1번째 청크 #########################
계와 가치사슬을 집중 조명하였으며, 10개 산업별로 
구체적인 비즈니스 활용 사례들을 정리하는 데 중점을 두었다. 특히, 금융분야의 JP 모건 사례,
헬스케어의 Roche 사례, 화학 분야의 BASF 사례, 물류산업의 DHL 사례는 눈여겨 볼 만 하다.
생성형 AI 도입이 초기 단계인 만큼, 사례가 제한적인 점, 그리고 적용사례의 성공여부가 아직 
판명되지 않은 점 등 제약적인 상황이지만 기업들이 어떻게 대응하고 있는지를 소개함으로써 AI의 
도입을 검토하고 있는 후발주자들에게 길라잡이가 되었으면 한다. 동시에 ‘game changer’로서의 
생성형 AI로 인해 기업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 
상상을 펼쳐볼 수 있으면 좋겠다.
들어가며

######################### 2번째 청크 #########################
계와 가치사슬을 집중 조명하였으며, 10개 산업별로 
구체적인 비즈니스 활용 사례들을 정리하는 데 중점을 두었다. 특히, 금융분야의 JP 모건 사례,
헬스케어의 Roche 사례, 화학 분야의 BASF 사례, 물류산업의 DHL 사례는 눈여겨 볼 만 하다.
생성형 AI 도입이 초기 단계인 만큼, 사례가 제한적인 점, 그리고 적용사례의 성공여부가 아직 
판명되지 않은 점 등 제약적인 상황이지만 기업들이 어떻게 대응하고 있는지를 소개함으로써 AI의 
도입을 검토하고 있는 후발주자들에게 길라잡이가 되었으면 한다. 동시에 ‘game changer’로서의 
생성형 AI로 인해 기업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 
상상을 펼쳐볼 수 있으면 좋겠다.
들어가며

######################### 3번째 청크 #########################
계와 가치사슬을 집중 조명하였으며, 10개 산업별로 
구체적인 비즈니스 활

In [12]:
# contextual_db = ContextualVectorDB('test_1')
# contextual_db.load_data(document)
contextual_results = contextual_db.search("금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘", k=5)
contextual_rag = []

for id, result in enumerate(contextual_results):
    print('#' * 25 + f' {id+1}번째 청크 ' + '#' * 25)
    print(f'{result['metadata']['chunk_content']}')
    contextual_rag.append(result['metadata']['chunk_content'])

######################### 1번째 청크 #########################
클라우드
컴퓨팅 하드웨어

######################### 2번째 청크 #########################
변수에 대응이 가능하고, 농가의 생산성과 에너지 효율을 극대화

######################### 3번째 청크 #########################
업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 
상상을 펼쳐볼 수 있으면 좋겠다.
들어가며

######################### 4번째 청크 #########################
결함, 오염 또는 불규칙성을 감지하여 고품질의 제품만 출시
•
풍미 및 레시피 최적화: 소비자 선호도와 트렌드를 분석하여 음식의 맛과 레시피를 최적화
•
레스토랑의 메뉴 맞춤화: 고객의 선호도, 식단 제한, 과거 주문 데이터를 기반으로 메뉴를 개인화, 식사 경험 
및 고객 만족도 개선

######################### 5번째 청크 #########################
반 조기 진단, 환자 
경험 개선
이용료
AI 
솔루션/
플랫폼
바이오테크 기업
AI/클라우드 플랫폼
신약 
개발
고객 세그먼트
의료 기관
(병원, 진료소, 약국)
실험실
(병원, 진료소)
의약품, 
진단기기 판매
제품 구매
의료 데이터 제공



In [13]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest'
)
query_embedding = genai.embed_content(
    model = 'models/text-embedding-004',
    content = "금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘"
)['embedding']

In [14]:
sim = np.dot(contextual_db.embeddings, query_embedding)
for sim_id in sim.argsort()[::-1][:10]:
    print(f'[{sim[sim_id]:.4f}] {contextual_db.metadata[sim_id]['chunk_content']}')
    print("#" * 20)

[0.9558] 클라우드
컴퓨팅 하드웨어

####################
[0.9165] 변수에 대응이 가능하고, 농가의 생산성과 에너지 효율을 극대화

####################
[0.9066] 업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 
상상을 펼쳐볼 수 있으면 좋겠다.
들어가며

####################
[0.8843] 결함, 오염 또는 불규칙성을 감지하여 고품질의 제품만 출시
•
풍미 및 레시피 최적화: 소비자 선호도와 트렌드를 분석하여 음식의 맛과 레시피를 최적화
•
레스토랑의 메뉴 맞춤화: 고객의 선호도, 식단 제한, 과거 주문 데이터를 기반으로 메뉴를 개인화, 식사 경험 
및 고객 만족도 개선

####################
[0.8769] 반 조기 진단, 환자 
경험 개선
이용료
AI 
솔루션/
플랫폼
바이오테크 기업
AI/클라우드 플랫폼
신약 
개발
고객 세그먼트
의료 기관
(병원, 진료소, 약국)
실험실
(병원, 진료소)
의약품, 
진단기기 판매
제품 구매
의료 데이터 제공

####################
[0.8761] 률
효율화
고객 경험
•
행정 업무 및 서류 검토 자동화 중심으로 활용 증
•
각종 문서 작성 및 고객 응대 서비스를 위해 AI 활용 검토 단계
*각 산업별 ’AI 도입 및 활용 수준’은 타 산업 대비 상대적 비교 기준
신약개발
기타 의료
산업 전반 
AI 활용 수준 下 
자동화/챗봇
맞춤 서비스
전체 평균 AI 도입 수준 

####################
[0.8730]  건강하게 생산할 수 있도록 지원
•
토양 건강 모니터링: 화학적 토양 분석을 수행하고 누락된 영양소를 정확하게 추정
•
농작물 보호: 식물의 상태를 모니터링하여 질병을 발견 및 예측, 잡초를 식별 및 제거
•
지능형 살포: 잡초나 해충 방제를 자동화. 농작물의 이력, 토양 상태 또는 작물 유형에 대한 데이터 분석을 
기반으로 각 밭에 필요한 농약의 양을 계산
•
챗봇

In [15]:
retrieve_results = retrieve_advanced(
    query = "금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘",
    db = contextual_db, 
    es_bm25=es_bm25,
    k = 5,
    semantic_weight=0.5,
    bm25_weight=0.5)

hybrid_rag = []
for id, result in enumerate(retrieve_results):
    print('#' * 25 + f' {id+1}번째 청크 ' + '#' * 25)
    print(f'{result['chunk'][0]['chunk_content']}')
    hybrid_rag.append(result['chunk'][0]['chunk_content'])

######################### 1번째 청크 #########################
계와 가치사슬을 집중 조명하였으며, 10개 산업별로 
구체적인 비즈니스 활용 사례들을 정리하는 데 중점을 두었다. 특히, 금융분야의 JP 모건 사례,
헬스케어의 Roche 사례, 화학 분야의 BASF 사례, 물류산업의 DHL 사례는 눈여겨 볼 만 하다.
생성형 AI 도입이 초기 단계인 만큼, 사례가 제한적인 점, 그리고 적용사례의 성공여부가 아직 
판명되지 않은 점 등 제약적인 상황이지만 기업들이 어떻게 대응하고 있는지를 소개함으로써 AI의 
도입을 검토하고 있는 후발주자들에게 길라잡이가 되었으면 한다. 동시에 ‘game changer’로서의 
생성형 AI로 인해 기업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 
상상을 펼쳐볼 수 있으면 좋겠다.
들어가며

######################### 2번째 청크 #########################
클라우드
컴퓨팅 하드웨어

######################### 3번째 청크 #########################
변수에 대응이 가능하고, 농가의 생산성과 에너지 효율을 극대화

######################### 4번째 청크 #########################
업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 
상상을 펼쳐볼 수 있으면 좋겠다.
들어가며

######################### 5번째 청크 #########################
42%의 여행자들이 AI가 생성해준 여행 일정을 선호한다고 조사되었다.
현재 AI는 ① 빅데이터 분석 기반의 여행지, 항공 및 호텔 예약과, ② 실시간 정보 탐색을 통한 맞춤 여행 
일정(타임라인) 제안, 그리고 ③ 여행 경험을 실시간으로 지원(정보 업데이트, 번역 등)하는 챗봇 기능 
중심으로 집중 활용되고 있다. 
[관광 산업 밸류체인 

In [16]:
def compare_rag_performance(query):
    contextual_results = contextual_db.search(query, k=5)
    contextual_rag = []
    for id, result in enumerate(contextual_results):
        # print('#' * 25 + f'{id+1}번째 청크' + '#' * 25)
        # print(f'{result['metadata']['chunk_content']}')
        contextual_rag.append(result['metadata']['chunk_content'])

    es = Elasticsearch("http://localhost:9200")
    es.indices.refresh(index=es_bm25.index_name)
    elasticsearch_results = es.search(
        index= es_bm25.index_name,
        body={"query": {"multi_match":
                        {"query": query,
                        "fields": ["chunk_content", "contextualized_content"]}
                        }})
    elastic_rag = []

    for id, result in enumerate(elasticsearch_results.body['hits']['hits']):
        # print('#' * 25 + f'{id+1}번째 청크' + '#' * 25)
        # print(f'{result['_source']['chunk_content']}')
        elastic_rag.append(result['_source']['chunk_content'])

    retrieve_results = retrieve_advanced(
        query = query,
        db = contextual_db, 
        es_bm25=es_bm25,
        k = 5,
        semantic_weight=0.5,
        bm25_weight=0.5)

    hybrid_rag = []
    for id, result in enumerate(retrieve_results):
        # print('#' * 25 + f'{id+1}번째 청크' + '#' * 25)
        # print(f'{result['chunk'][0]['chunk_content']}')
        hybrid_rag.append(result['chunk'][0]['chunk_content'])

    contextual_rag_context = "\n\n".join(contextual_rag)
    elastric_rag_context = "\n\n".join(elastic_rag)
    hybrid_rag_context = "\n\n".join(hybrid_rag)

    check_rag_list = [contextual_rag_context, elastric_rag_context, hybrid_rag_context]
    rag_list = ["Contextual Retrieval", "ElasticSearch Retrieval", "Hybrid Retrieval"]

    for rag, context in zip(rag_list, check_rag_list):
        prompt = f""" 
        Using only the CONTEXT below, provide an accurate answer to the QUESTION in Korean (Hangul).

        Only use information provided in the CONTEXT for your answer.
        Do not reference or incorporate any external information.
        If the answer cannot be determined based on the CONTEXT, respond with "답변이 어렵습니다."
        CONTEXT: {context}

        QUESTION: {query}

        ANSWER:
        """
        client = genai.GenerativeModel(
            model_name='gemini-1.5-flash-latest'
        )
        response = client.generate_content(contents=prompt).text
        print("#" * 40)
        print(f"{rag}를 통한 RAG 결과")
        print("-" * 40)
        print(f"답변 : {response}\n")

In [17]:
compare_rag_performance("금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘")

########################################
Contextual Retrieval를 통한 RAG 결과
----------------------------------------
답변 : 답변이 어렵습니다. 


########################################
ElasticSearch Retrieval를 통한 RAG 결과
----------------------------------------
답변 : 답변이 어렵습니다. 


########################################
Hybrid Retrieval를 통한 RAG 결과
----------------------------------------
답변 : 답변이 어렵습니다. 




In [18]:
contextual_db.metadata

[{'doc_id': 'doc_1',
  'chunk_id': 'doc_1_chunk_1',
  'chunk_content': 'Samil PwC  l  Samil Insight\n0\n',
  'contextualized_content': '삼일 PwC와 삼일 인사이트의 제목입니다. \n'},
 {'doc_id': 'doc_2',
  'chunk_id': 'doc_2_chunk_1',
  'chunk_content': 'Samil PwC  l  Samil Insight\n1\n2022년 11월 OpenAI의 ChatGPT 출시 이후 엄청난 속도로 생성형 AI 산업이 전방위적으로 \n확장되고 있으며, 생성형 AI의 기능, 한계, 그에 대한 대책 등 다양한 분석이 이어지고 있다.\n다만, 총론적인 보고서들은 이미 많이 발표되고 있지만 산업별, 기업별로 AI가 구체적으로 어떻게 \n활용되고 있는지에 대한 적용 사례에 대한 보고는 아직 미진하다. 아마도 생성형 AI의 초기 단계이기 \n때문에 매일 다양한 시도가 나타났다 사라지고 있고 적용사례의 성공여부도 판가름하기 어려워 이를 \n체계적으로 정리하기가 만만치 않은 점이 한 이유일 수 있을 것이다. 하지만, 각 기업들은 AI를 도입할 \n것인가? 어느 정도 도입할 것인가? 어떤 방식으로 도입할 ',
  'contextualized_content': '이 텍스트는 보고서의 서론 부분으로 생성형 AI 산업의 급속한 성장과 더불어 이 보고서의 목적과 필요성을 설명하고 있습니다. \n'},
 {'doc_id': 'doc_2',
  'chunk_id': 'doc_2_chunk_2',
  'chunk_content': ' 만만치 않은 점이 한 이유일 수 있을 것이다. 하지만, 각 기업들은 AI를 도입할 \n것인가? 어느 정도 도입할 것인가? 어떤 방식으로 도입할 것인가? 등에 대한 의사결정을 할 수 밖에 \n없는 단계에 있으며 이러한 결정에 앞서 동종업계든 이종업계든 다른 기업들이 어떻게 이를 시도하고 \n적용하고 있는지를 아

In [19]:
from typing import List, Dict, Tuple
class LLM_Reranker:
    def __init__(self):
        self.client = genai.GenerativeModel(
            model_name = 'gemini-1.5-flash-latest'
        )

    def rerank_results(self, query: str, results: List[Dict], k: int = 5) -> List[Dict]:
        # Prepare the summaries with their indices
        summaries = []
        # print(len(results))

        for i, result in enumerate(results):
            summary = f"[{i}] Document Summary: {result['chunk'][0]['contextualized_content']}"
            summaries.append(summary)
        joined_summaries = "\n\n".join(summaries)
        
        prompt = f"""
        Query: {query}
        You are about to be given a group of documents, each preceded by its index number in square brackets. Your task is to select the only {k} most relevant documents from the list to help us answer the query.
        
        <documents>
        {joined_summaries}
        </documents>

        Output only the indices of {k} most relevant documents in order of relevance, separated by commas, enclosed in XML tags here:
        <relevant_indices>put the numbers of your indices here, seeparted by commas</relevant_indices>
        """
        try:
            try:
                response = self.client.generate_content(
                    contents=prompt,
                    generation_config=genai.GenerationConfig(
                        stop_sequences=["</relevant_indices>"],
                        temperature=0        
                    )
                ).text.strip()
            except:
                time.sleep(60)
                print(f'Time sleep for 1 Minute (Rerank)')
                response = self.client.generate_content(
                        contents=prompt,
                        generation_config=genai.GenerationConfig(
                        stop_sequences=["</relevant_indices>"],
                        temperature=0        
                    )
                ).text.strip()
            # Extract the indices from the response
            indices_str = response
            relevant_indices = []
            for idx in indices_str.split('>')[-1].split(','):
                try:
                    relevant_indices.append(int(idx.strip()))
                except ValueError:
                    continue  # Skip invalid indices
            # print(indices_str)
            # print(relevant_indices)
            # If we didn't get enough valid indices, fall back to the top k by original order
            if len(relevant_indices) == 0:
                relevant_indices = list(range(min(k, len(results))))
            
            # Ensure we don't have out-of-range indices
            relevant_indices = [idx for idx in relevant_indices if idx < len(results)]
            
            # Return the reranked results
            reranked_results = [results[idx] for idx in relevant_indices[:k]]
            # Assign descending relevance scores
            for i, result in enumerate(reranked_results):
                result['relevance_score'] = 100 - i  # Highest score is 100, decreasing by 1 for each rank
            
            return reranked_results
        
        except Exception as e:
            print(f"An error occurred during reranking: {str(e)}")
            # Fall back to returning the top k results without reranking
            return results[:k]

    def retrieve_advanced(
            self, 
            query: str, 
            contextual_db: ContextualVectorDB, 
            es_bm25 = ElasticsearchBM25, 
            k: int = 5, 
            initial_k: int = 50
            ) -> Tuple[List[Dict], str]:
        # Step 1: Get initial results
        # initial_results = db.search(query, k=initial_k)
        initial_results = retrieve_advanced(query=query, db=contextual_db, es_bm25=es_bm25, k=initial_k, semantic_weight=0.5, bm25_weight=0.5)

        # Step 2: Re-rank results
        reranked_results = self.rerank_results(query, initial_results, k=k)
        
        # Step 3: Generate new context string from re-ranked results
        new_context = ""
        for result in reranked_results:
            chunk = result['metadata']
            new_context += f"\n <document> \n {chunk['chunk_heading']}\n\n{chunk['text']} \n </document> \n"
        
        return reranked_results, new_context

    # The answer_query_advanced function remains unchanged
    def answer_query_advanced(self, query: str, db: ContextualVectorDB):
        documents, context = retrieve_advanced(query, db)
        prompt = f"""
        You have been tasked with helping us to answer the following query: 
        <query>
        {query}
        </query>
        You have access to the following documents which are meant to provide context as you answer the query:
        <documents>
        {context}
        </documents>
        Please remain faithful to the underlying context, and only deviate from it if you are 100% sure that you know the answer already. 
        Answer the question now, and avoid providing preamble such as 'Here is the answer', etc
        """
        try:
            response = self.client.generate_content(
                        contents=prompt).text
        except:
            time.sleep(60)
            print(f'Time sleep for 1 Minute')
            response = self.client.generate_content(
                        contents=prompt).text
        return response

In [20]:
retrieve_results = retrieve_advanced(
    query = "금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘",
    db = contextual_db, 
    es_bm25=es_bm25,
    k = 50,
    semantic_weight=0.5,
    bm25_weight=0.5)

In [21]:
retrieve_results

[{'chunk': [{'doc_id': 'doc_2',
    'chunk_id': 'doc_2_chunk_3',
    'chunk_content': '계와 가치사슬을 집중 조명하였으며, 10개 산업별로 \n구체적인 비즈니스 활용 사례들을 정리하는 데 중점을 두었다. 특히, 금융분야의 JP 모건 사례,\n헬스케어의 Roche 사례, 화학 분야의 BASF 사례, 물류산업의 DHL 사례는 눈여겨 볼 만 하다.\n생성형 AI 도입이 초기 단계인 만큼, 사례가 제한적인 점, 그리고 적용사례의 성공여부가 아직 \n판명되지 않은 점 등 제약적인 상황이지만 기업들이 어떻게 대응하고 있는지를 소개함으로써 AI의 \n도입을 검토하고 있는 후발주자들에게 길라잡이가 되었으면 한다. 동시에 ‘game changer’로서의 \n생성형 AI로 인해 기업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 \n상상을 펼쳐볼 수 있으면 좋겠다.\n들어가며\n',
    'contextualized_content': '본 보고서는 생성형 AI가 산업별로 어떻게 활용되고 있는지 구체적인 국내외 사례를 통해 분석하고, 그를 통해 기업들이 AI를 도입하는 데 있어 도움을 주는 것을 목표로 합니다. \n'}],
  'score': 1.0,
  'from_semantic': True,
  'from_bm25': True},
 {'chunk': [{'doc_id': 'doc_9',
    'chunk_id': 'doc_9_chunk_5',
    'chunk_content': '클라우드\n컴퓨팅 하드웨어\n',
    'contextualized_content': 'AI 생태계 내 하드웨어 및 인프라 부분을 설명하는 그림의 맨 아래에 위치하는 부분입니다. \n'}],
  'score': 0.5,
  'from_semantic': True,
  'from_bm25': False},
 {'chunk': [{'doc_id': 'doc_46',
    'chunk_id': 'do

In [22]:
reranker = LLM_Reranker()
rerank_results = reranker.rerank_results(query="금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘", results = retrieve_results, k = 5)

In [23]:
rerank_rag = []
for id, result in enumerate(rerank_results):
    print('#' * 25 + f' {id+1}번째 청크 ' + '#' * 25)
    print(f'{result['chunk'][0]['chunk_content']}')
    rerank_rag.append(result['chunk'][0]['chunk_content'])

######################### 1번째 청크 #########################
Samil PwC  l  Samil Insight
17
1-1. 금융: 비즈니스 AI 활용 현황
III. 산업별 비즈니스 AI 활용 현황
금융업은 헬스케어,자동차 산업에 이어 AI영향력이 큰 업종 중 하나로,금융권의 수치,재무 등 방대한 정형 
데이터를 기반으로 강력한 AI비즈니스 창출이 가능한 산업이다.
특히 ① 신속한 고객 문의 대응을 위한 챗봇/상담봇, ② 투자성향, 리스크 수준, 기대 수익률 등 자산관리 및 
상품추천 정보를 제공하는 로보어드바이저(Robo-advisor), ③ 각종 서류를 자동으로 분석/분류하는 등 
프로세스를 개선해주는 로봇 프로세스 자동화(Robotic Process Automation, RPA) 기능에 
금융기관들은 주목하고 있다.
현재 국내외 금융기관들의 생성형 AI 도
######################### 2번째 청크 #########################
동화(Robotic Process Automation, RPA) 기능에 
금융기관들은 주목하고 있다.
현재 국내외 금융기관들의 생성형 AI 도입 및 활용 방향성을 비추어 봤을때 특히 ‘마케팅/영업’과 
‘고객관리’와 같이 개별 고객에 대한 정밀한 분석 기반의 1대1 맞춤형 서비스가 요구되는 영역을 중심으로 
보다 고도화된 생성형 AI기술을 활용하고자 하는 니즈가 높아지고 있다.
[금융 산업 밸류체인 -AI 적용 예시]
마케팅, 영업
상품, 서비스
고객관리
운영 관리
리스크 관리
• 초개인화 고객 분석
• 시장 분석/예측
• 원격 지원
• 잠재 고객 발굴
• 챗봇
• 고객 segment별 
특화된 맞춤상품 추천
• 신원인식(얼굴, 음성)
• 컨설팅(투자, 보험, 
절세 등)
• 금융/비금융 연계 
종합 플랫
######################### 3번째 청크 #########################
태로 사용할 수 있는 ‘AI 신용
평가

In [24]:
rag = "Contextual + (Semantic + BM25) + Reranker"
context = "\n\n".join(rerank_rag)
query = "금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘. 최대한 상세하게 설명해줘."

prompt = f""" 
Using only the CONTEXT below, provide an accurate answer to the QUESTION in Korean (Hangul).

Only use information provided in the CONTEXT for your answer.
Do not reference or incorporate any external information.
If the answer cannot be determined based on the CONTEXT, respond with "답변이 어렵습니다."
CONTEXT: {context}

QUESTION: {query}

ANSWER:
"""
client = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest'
)
response = client.generate_content(contents=prompt).text
print("#" * 40)
print(f"{rag}를 통한 RAG 결과")
print("-" * 40)
print(f"답변 : {response}\n")

########################################
Contextual + (Semantic + BM25) + Reranker를 통한 RAG 결과
----------------------------------------
답변 : 금융권에서는 생성형 AI를 주로 마케팅/영업과 고객관리 분야에서 활용하고 있습니다. 특히 개별 고객에 대한 정밀한 분석을 기반으로 1대1 맞춤형 서비스를 제공하는 데 유용하게 활용되고 있습니다.  

예를 들어, 생성형 AI는 고객의 투자 성향, 리스크 수준, 기대 수익률 등을 분석하여 개인 맞춤형 상품 추천, 자산 관리 컨설팅, 금융/비금융 연계 종합 플랫폼 서비스를 제공할 수 있습니다. 또한, 고객 문의에 대한 신속하고 정확한 답변을 제공하는 챗봇, 고객의 신원을 인식하는 얼굴/음성 인식 시스템, 그리고 금융 사기 탐지 및 예방에도 활용될 수 있습니다.

생성형 AI는 고객에게 보다 개인화된 서비스를 제공하고 효율성을 높이는 데 크게 기여할 것으로 예상됩니다. 




In [25]:
from IPython.display import Markdown
Markdown(response)

금융권에서는 생성형 AI를 주로 마케팅/영업과 고객관리 분야에서 활용하고 있습니다. 특히 개별 고객에 대한 정밀한 분석을 기반으로 1대1 맞춤형 서비스를 제공하는 데 유용하게 활용되고 있습니다.  

예를 들어, 생성형 AI는 고객의 투자 성향, 리스크 수준, 기대 수익률 등을 분석하여 개인 맞춤형 상품 추천, 자산 관리 컨설팅, 금융/비금융 연계 종합 플랫폼 서비스를 제공할 수 있습니다. 또한, 고객 문의에 대한 신속하고 정확한 답변을 제공하는 챗봇, 고객의 신원을 인식하는 얼굴/음성 인식 시스템, 그리고 금융 사기 탐지 및 예방에도 활용될 수 있습니다.

생성형 AI는 고객에게 보다 개인화된 서비스를 제공하고 효율성을 높이는 데 크게 기여할 것으로 예상됩니다. 


In [32]:
rerank_inputs = [(query, result['chunk'][0]['chunk_content']) for result in retrieve_results]
# rerank_inputs

In [85]:
import gc
gc.collect()

7

In [78]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# model_name = "'BAAI/bge-reranker-v2-m3'"
model_name = "Dongjin-kr/ko-reranker"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]

with torch.no_grad():
    inputs = tokenizer(rerank_inputs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\LeeSeungYong\langchain\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LeeSeungYong\.cache\huggingface\hub\models--Dongjin-kr--ko-reranker. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tensor([ 3.9907, -9.8244, -9.8245, -9.8247, -2.9951, -9.8244, -9.8163, -9.1662,
        -3.9930, -4.3214, -9.8240, -8.2679,  3.9325, -8.2192, -3.3102, -2.4136,
        -7.4922, -8.2801, -5.0900, -7.3276, -8.6552, -2.1247, -6.9939,  7.4474,
        -3.3513, -4.1066,  0.2785, -6.7167, -9.7822,  1.4517, -5.3458, -4.4951,
        -9.8067, -4.7634, -8.0718, -5.7102, -7.3095, -7.8747, -2.6332, -0.3856,
        -6.7883, -1.2419, -4.9321, -3.3251, -2.8604, -6.1635, -7.9117, -9.6421,
        -8.8281, -9.8176])


In [79]:
relevant_index_high = scores.numpy().argsort()[::-1]
relevant_index_high

import numpy as np
def sigmoid(x):
    return 1/(1+np.exp(-x))

relevance_scores = list(map(sigmoid, scores.numpy()))

reranked_results = [
    {
        "rerank_content" : rerank_inputs[idx][-1],
        "rerank_socre" : relevance_scores[idx],
        }
        for idx in relevant_index_high
        ]

In [80]:
baai_rag = [result['rerank_content'] for result in reranked_results[:5]]
sum([baai == rerank for rerank in rerank_rag for baai in baai_rag])

2

In [81]:
rerank_rag

['Samil PwC  l  Samil Insight\n17\n1-1. 금융: 비즈니스 AI 활용 현황\nIII. 산업별 비즈니스 AI 활용 현황\n\uf06c금융업은 헬스케어,자동차 산업에 이어 AI영향력이 큰 업종 중 하나로,금융권의 수치,재무 등 방대한 정형 \n데이터를 기반으로 강력한 AI비즈니스 창출이 가능한 산업이다.\n\uf06c특히 ① 신속한 고객 문의 대응을 위한 챗봇/상담봇, ② 투자성향, 리스크 수준, 기대 수익률 등 자산관리 및 \n상품추천 정보를 제공하는 로보어드바이저(Robo-advisor), ③ 각종 서류를 자동으로 분석/분류하는 등 \n프로세스를 개선해주는 로봇 프로세스 자동화(Robotic Process Automation, RPA) 기능에 \n금융기관들은 주목하고 있다.\n\uf06c현재 국내외 금융기관들의 생성형 AI 도',
 '동화(Robotic Process Automation, RPA) 기능에 \n금융기관들은 주목하고 있다.\n\uf06c현재 국내외 금융기관들의 생성형 AI 도입 및 활용 방향성을 비추어 봤을때 특히 ‘마케팅/영업’과 \n‘고객관리’와 같이 개별 고객에 대한 정밀한 분석 기반의 1대1 맞춤형 서비스가 요구되는 영역을 중심으로 \n보다 고도화된 생성형 AI기술을 활용하고자 하는 니즈가 높아지고 있다.\n[금융 산업 밸류체인 -AI 적용 예시]\n마케팅, 영업\n상품, 서비스\n고객관리\n운영 관리\n리스크 관리\n• 초개인화 고객 분석\n• 시장 분석/예측\n• 원격 지원\n• 잠재 고객 발굴\n• 챗봇\n• 고객 segment별 \n특화된 맞춤상품 추천\n• 신원인식(얼굴, 음성)\n• 컨설팅(투자, 보험, \n절세 등)\n• 금융/비금융 연계 \n종합 플랫',
 "태로 사용할 수 있는 ‘AI 신용\n평가모델'을 개발 중\n자산관리\n크래프트테크놀로지스\n•\n국내외 글로벌 금융기관들에 AI 기반 투자 전략 솔루션을 제공\n씨앤테크\n•\nAI 및 빅데이터 기반 365일 24시간 글로벌 자산관제서비스를 제

In [87]:
baai_rag

['동화(Robotic Process Automation, RPA) 기능에 \n금융기관들은 주목하고 있다.\n\uf06c현재 국내외 금융기관들의 생성형 AI 도입 및 활용 방향성을 비추어 봤을때 특히 ‘마케팅/영업’과 \n‘고객관리’와 같이 개별 고객에 대한 정밀한 분석 기반의 1대1 맞춤형 서비스가 요구되는 영역을 중심으로 \n보다 고도화된 생성형 AI기술을 활용하고자 하는 니즈가 높아지고 있다.\n[금융 산업 밸류체인 -AI 적용 예시]\n마케팅, 영업\n상품, 서비스\n고객관리\n운영 관리\n리스크 관리\n• 초개인화 고객 분석\n• 시장 분석/예측\n• 원격 지원\n• 잠재 고객 발굴\n• 챗봇\n• 고객 segment별 \n특화된 맞춤상품 추천\n• 신원인식(얼굴, 음성)\n• 컨설팅(투자, 보험, \n절세 등)\n• 금융/비금융 연계 \n종합 플랫',
 '계와 가치사슬을 집중 조명하였으며, 10개 산업별로 \n구체적인 비즈니스 활용 사례들을 정리하는 데 중점을 두었다. 특히, 금융분야의 JP 모건 사례,\n헬스케어의 Roche 사례, 화학 분야의 BASF 사례, 물류산업의 DHL 사례는 눈여겨 볼 만 하다.\n생성형 AI 도입이 초기 단계인 만큼, 사례가 제한적인 점, 그리고 적용사례의 성공여부가 아직 \n판명되지 않은 점 등 제약적인 상황이지만 기업들이 어떻게 대응하고 있는지를 소개함으로써 AI의 \n도입을 검토하고 있는 후발주자들에게 길라잡이가 되었으면 한다. 동시에 ‘game changer’로서의 \n생성형 AI로 인해 기업들의 비즈니스 모델이 향후 얼마나 변해 있을지에 대해 이 보고서를 통해서 각자 \n상상을 펼쳐볼 수 있으면 좋겠다.\n들어가며\n',
 'Samil PwC  l  Samil Insight\n17\n1-1. 금융: 비즈니스 AI 활용 현황\nIII. 산업별 비즈니스 AI 활용 현황\n\uf06c금융업은 헬스케어,자동차 산업에 이어 AI영향력이 큰 업종 중 하나로,금융권의 수치,재무 등 방대한 정형 \n데이터를 기반으로 강력한 

In [88]:
rag = "Contextual + (Semantic + BM25) + Reranker(BAAI)"
context = "\n\n".join(baai_rag)
query = "금융권에서 생성형AI가 활용되는 사례에 대해서 설명해줘. 최대한 상세하게 설명해줘."

prompt = f""" 
Using only the CONTEXT below, provide an accurate answer to the QUESTION in Korean (Hangul).

Only use information provided in the CONTEXT for your answer.
Do not reference or incorporate any external information.
CONTEXT: {context}

QUESTION: {query}

ANSWER:
"""
client = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest'
)
response = client.generate_content(contents=prompt).text
print("#" * 40)
print(f"{rag}를 통한 RAG 결과")
print("-" * 40)
print(f"답변 : {response}\n")

########################################
Contextual + (Semantic + BM25) + Reranker(BAAI)를 통한 RAG 결과
----------------------------------------
답변 : 금융권에서는 생성형 AI를 활용하여 고객에게 1대1 맞춤형 서비스를 제공하고 있습니다. 특히 마케팅/영업과 고객관리 분야에서 생성형 AI 기술을 활용하여 고객 분석, 상품 추천, 챗봇 등을 통해 고객 만족도를 높이고 있습니다. 

예를 들어, 생성형 AI는 고객의 투자 성향, 리스크 수준, 기대 수익률 등을 분석하여 개인 맞춤형 자산 관리 및 상품 추천 서비스를 제공하는 로보어드바이저(Robo-advisor)에 활용될 수 있습니다. 또한, 고객 문의에 신속하게 응답하는 챗봇/상담봇으로 활용되어 고객 만족도를 높일 수 있습니다. 

뿐만 아니라, 생성형 AI는 금융 문서 분석, 위험 관리, 사기 방지 등 다양한 분야에서 활용되어 업무 효율성을 높이고 있습니다. 




In [96]:
rerank_context = "\n\n".join(rerank_rag)
baai_context = "\n\n".join(baai_rag)

prompt = f"""
너는 최고의 논리전문가이자 작문가야. 
아래 2개의 문맥(A Context, B Context)이 있는데, 
2개의 문맥 중 질문(query)에 대한 답변을 하기에 참고하기 더 적절한 문맥이 무엇인지 
선택해줘 답변은 A Context와 B Context 둘 중에서 하나를 고르고,
해당 문맥을 선택한 이유를 설명해주면돼(선택되지 않은 문맥보다 더 적합한 이유를 설명해줘)

A Context : {rerank_context}

B Context : {baai_context}

Question : {query}
"""

client = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest'
)
response = client.generate_content(
    contents=prompt,
    generation_config = genai.GenerationConfig(
        temperature = 0
    )).text

In [97]:
Markdown(response)

답변: **A Context**가 질문에 대한 답변을 하기에 더 적절합니다.

**이유:**

* **A Context**는 금융 산업에서 생성형 AI가 활용되는 구체적인 사례와 함께, 밸류체인별 적용 예시를 제시하고 있습니다. 특히, 마케팅/영업, 고객관리, 자산관리 등 다양한 분야에서 생성형 AI가 어떻게 활용될 수 있는지 상세하게 설명하고 있습니다. 
* **B Context**는 생성형 AI 도입 초기 단계임을 언급하며, 금융 산업에서의 활용 사례를 구체적으로 다루기보다는, 생성형 AI가 금융 산업에 미칠 수 있는 잠재적 영향과 데이터 활용의 중요성을 강조하고 있습니다. 

따라서 질문에서 요구하는 "금융권에서 생성형 AI가 활용되는 사례"에 대한 상세한 설명은 **A Context**에서 더 잘 제공됩니다. **A Context**는 챗봇/상담봇, 로보어드바이저, RPA 등 구체적인 생성형 AI 활용 사례와 함께, 각 사례가 어떤 방식으로 금융 서비스를 개선하는지 설명하고 있습니다. 
